In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import copy
import tqdm
import glob
import keras
import pickle
import logging
import pandas as pd
import tensorflow as tf
from datetime import datetime

In [2]:
logger = tf.get_logger()
logger.setLevel(logging.ERROR) # or logging.INFO, logging.WARNING, etc.

In [3]:
ROOT_S1 = "_intermediate/stage1_pathology_masked"
INT_S1_DIR = sorted(glob.glob(os.path.join(ROOT_S1, "*")))[-1]
print(INT_S1_DIR)
INT_S2_DIR = f"{ROOT_S1.replace("1", "2")}/{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
print(INT_S2_DIR)

#DATASET_DIR = "_data/plant_pathology"
DATASET_DIR = "_data/plantdoc_csv"

PROB_THRESHOLD = .5

_intermediate/stage1_pathology_masked/2024_11_24_16_39_12
_intermediate/stage2_pathology_masked/2024_11_25_10_29_26


In [4]:
os.makedirs(INT_S2_DIR, exist_ok=True)

In [5]:
train_data = pd.read_csv(os.path.join(DATASET_DIR, "data.csv"))

In [6]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [7]:
stage1_names = [d for d in os.listdir(INT_S1_DIR) if os.path.isdir(os.path.join(INT_S1_DIR, d)) and d != "patches"]

In [8]:
stage1_names

['SAM + ResNet', 'YOLOv8', 'SAM + YOLOv8']

In [9]:
stage2_names = ["InceptionV3", "VGG19", "AlexNet", "ResNet152V2"]

In [10]:
# Load best performing model by type
stage2_dict = {
    name: keras.models.load_model(f"../disease_detection/out/best{name}_rgb.keras")
    for name in stage2_names
}

In [13]:
import numpy as np

BATCH_SIZE = 16  # Adjust this value based on your GPU memory and model size
PADDING = 25
# paddings as top right bottom left
PADDINGS = (PADDING, PADDING, PADDING, PADDING)
stage2_results = {}

for stage2_name, stage2_model in stage2_dict.items():
    for stage1_name in stage1_names:
        with open(os.path.join(INT_S1_DIR, stage1_name, "data.pkl"), "rb") as file:
            stage1_result = pickle.load(file)
        stage_name = f'({stage1_name}) + ({stage2_name})'
        print(f'Evaluating {stage_name}')
        stage2_result = copy.deepcopy(stage1_result)
        del stage1_result
        #stage2_results[stage_name] = copy.deepcopy(stage1_result)

        # Iterate through the results in batches
        for index, data in tqdm.tqdm(stage2_result.items(), desc=stage_name):
            # Extract the 'healthy' status
            healthy = train_data.iloc[index]['healthy']

            # Collect patches for the current index
            masks = data['masks']
            patches = [mask['patch'] for mask in masks]

            resized_patches = []
            for patch in patches:
                padding = tf.constant([[PADDINGS[0], PADDINGS[2]], [PADDINGS[1], PADDINGS[3]], [0, 0]])
                # Apply padding
                padded_patch = tf.pad(patch, paddings=padding, mode='CONSTANT', constant_values=0)
                updated_patch = tf.image.resize(padded_patch, (224, 224))
                new_color = tf.constant([.5, .5, .5], dtype=tf.float32)

                # Create a mask for black pixels
                black_pixel_mask = tf.reduce_all(updated_patch == [0.0, 0.0, 0.0], axis=-1)  # Shape: (height, width)

                # Expand mask to match the image dimensions
                mask_expanded = tf.expand_dims(black_pixel_mask, axis=-1)  # Shape: (height, width, 1)

                # Replace black pixels with the new color
                updated_patch = tf.where(mask_expanded, new_color, updated_patch)
                
                resized_patches.append(updated_patch)
                
            batch_inputs = tf.stack(resized_patches)

            # Convert resized patches into a tensor
            #batch_inputs = tf.convert_to_tensor(resized_patches, dtype=tf.float32)

            batch_inputs = tf.expand_dims(batch_inputs, axis=0) if len(batch_inputs.shape) == 3 else batch_inputs
            
            # Process in mini-batches
            num_patches = len(patches)
            batch_predictions = []
            for start in range(0, num_patches, BATCH_SIZE):
                end = start + BATCH_SIZE
                batch = batch_inputs[start:end]
                predictions = stage2_model(batch).numpy()
                batch_predictions.extend(predictions)

            # Update the predictions and mask probabilities
            data['predictions'] = []
            for mask, prob in zip(masks, batch_predictions):
                prob_diseased = prob.squeeze()[1]  # Assuming second index is for 'diseased'
                data['predictions'].append(prob_diseased)
                mask['prob_diseased'] = prob_diseased
                mask['predicted_sick'] = int(prob_diseased > PROB_THRESHOLD)
        stage_dir = os.path.join(INT_S2_DIR, stage_name)
        os.makedirs(stage_dir, exist_ok=True)
        with open(os.path.join(stage_dir, "data.pkl"), "wb+") as file:
            pickle.dump(stage2_result, file, protocol=pickle.HIGHEST_PROTOCOL)
        del stage2_result

Evaluating (SAM + ResNet) + (InceptionV3)


(SAM + ResNet) + (InceptionV3):   0%|          | 0/512 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
W0000 00:00:1732527116.374292 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.401723 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.402210 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.451516 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.451984 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.452442 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732527116.452893 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0

Evaluating (YOLOv8) + (InceptionV3)


(SAM + YOLOv8) + (InceptionV3): 100%|██████████| 512/512 [05:20<00:00,  1.60it/s]


Evaluating (SAM + ResNet) + (VGG19)


...

(SAM + ResNet) + (ResNet152V2):   0%|          | 0/512 [00:00<?, ?it/s]W0000 00:00:1732528448.594014 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.594740 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.595357 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.595964 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.596693 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.597435 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.598200 3560463 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732528448.599088 3560463 gpu_timer.cc:114] Skipping the delay kernel, measur

Evaluating (YOLOv8) + (ResNet152V2)


(YOLOv8) + (ResNet152V2): 100%|██████████| 512/512 [03:07<00:00,  2.73it/s]


Evaluating (SAM + YOLOv8) + (ResNet152V2)


(SAM + YOLOv8) + (ResNet152V2): 100%|██████████| 512/512 [10:24<00:00,  1.22s/it]


In [ ]:
with open(os.path.join(INT_S2_DIR, "total.pkl"), "wb+") as file:
    pickle.dump(stage2_results, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for name, data in stage2_results.items():
    os.makedirs(os.path.join(INT_S2_DIR, name), exist_ok=True)
    with open(os.path.join(INT_S2_DIR, name, "data.pkl"), 'wb+') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(stage2_results.keys())

first_key1 = next(iter(stage2_results.keys()))
print(stage2_results[first_key1].keys())

first_key2 = next(iter(stage2_results[first_key1].keys()))
print(stage2_results[first_key1][first_key2]['masks'][0].keys())